In [1]:
import os, json, gc
import pandas as pd
import numpy as np
from scipy.spatial import distance
from scipy.stats import pearsonr
from urllib2 import Request, urlopen
from pandas.io.json import json_normalize

import gc

SOCIAL_INTERESTS_URI = "http://0.0.0.0:3000/api/v4/re/interests/lifestyle"
EVENT_TYPES_URI = "http://0.0.0.0:3000/api/v4/re/events/types"

def _social_interests_data():
    request=Request(SOCIAL_INTERESTS_URI)
    profiles = json.loads(urlopen(request).read())
    df = pd.DataFrame(profiles)
    return df

def _event_types_data():
    request=Request(EVENT_TYPES_URI)
    events = json.loads(urlopen(request).read())
    df = pd.DataFrame(events)
    return df

    
def _index_of(id, matrix):
    if not matrix[matrix.account_id == id].empty:
        print matrix[matrix.account_id == id]
        return matrix[matrix.account_id == id].index.tolist()[0]
    else:
        print "cannot find acccount"
# print social_interests_data()
# print event_types_data()

In [ ]:
def _matrix_with_loc(location):
    df_profile = _social_interests_data()
    print df_profile.shape
    df_profile[['account_id']] = df_profile[['account_id']].astype(int)
    df_profile['location'].fillna('empty', inplace=True)
    df_profile = pd.pivot_table(df_profile, index=['account_id', 'location'], columns='social', values='indicator')
    print df_profile.shape
    df_profile_t = df_profile.reset_index()
    df_profile_t = df_profile_t.fillna(value=0)
    df_profile_t = df_profile_t[(df_profile_t.location == '') | (df_profile_t.location == 'empty') |(df_profile_t.location.str.contains(location))]
    print(df_profile_t)

    df_event = _event_types_data()
    print df_event.shape
    df_event[['account_id']] = df_event[['account_id']].astype(int)
    df_event[['count']] = df_event[['count']].astype(int)
    df_event[['event_type']] = df_event[['event_type']].astype(int)
    df_event = pd.pivot_table(df_event, index=['account_id', 'email'], columns='event_type', values='count')
#     df_event = df_event[df_event.columns[2:]].sort_index(axis=1, ascending=True)
    print df_event.shape
    df_event_t = df_event.reset_index()
    df_event_t = df_event_t.fillna(value=0)
    print(df_event_t.shape)
#     print df_event_t
    
    print "------"

    df_new = df_event_t.set_index('account_id').join(df_profile_t.set_index('account_id'))
    print df_new
    df_new = df_new.dropna()
    df_new = df_new.sort_values(by='email', ascending=False)
    df_new = df_new.reset_index()
    print df_new.shape
    print "---------"
    print df_new[df_new.account_id == 41143].index.tolist()[0]
#     return df_new

_matrix_with_loc("New York")

In [2]:
def _interests_matrix_with_loc(location):
    df_profile = _social_interests_data()
    df_profile[['account_id']] = df_profile[['account_id']].astype(int)
    df_profile['location'].fillna('empty', inplace=True)
    df_profile_t = pd.pivot_table(df_profile, index=['account_id', 'location'], columns=['social'], values='indicator')
    del df_profile
    gc.collect()
    
    df_profile_t = df_profile_t.reset_index()
    df_profile_t = df_profile_t.fillna(value=0)
    df_profile_t = df_profile_t[(df_profile_t.location == '') | (df_profile_t.location == 'empty') | (df_profile_t.location.str.contains(location))]
    
    df_profile_f = df_profile_t.dropna()
    del df_profile_t
    gc.collect()
    
    df_profile_s = df_profile_f.sort_values(by='account_id', ascending=True)
    del df_profile_f
    gc.collect()
    
    df_profile_s.reset_index(drop=True, inplace=True)
    
    return df_profile_s

# interests_matrix_with_loc('New York')

In [ ]:
def _events_matrix_with_loc(location):
    df_event = _event_types_data()
    df_event[['account_id']] = df_event[['account_id']].astype(int)
    df_event[['count']] = df_event[['count']].astype(int)
    df_event[['event_type']] = df_event[['event_type']].astype(int)
    
    df_event_t = pd.pivot_table(df_event, index=['account_id', 'email'], columns='event_type', values='count')
    del df_event
    gc.collect()
    
    df_event_t.reset_index(inplace=True)
    df_event_t = df_event_t.fillna(value=0)
    df_event_s = df_event_t.sort_values(by='account_id', ascending=True)
    del df_event_t
    gc.collect()
    
    df_event_s.reset_index(drop=True, inplace=True)
#     print df_event_s[df_event_s.account_id == 40383]
#     print df_event_s[df_event_s.account_id == 28071]
    return df_event_s


In [3]:
def interests_sim_with_loc(id, location):
    # df_new = _matrix_with_loc(location)
    # df_profile_t = df_new[df_new.columns[14:]]
    df_new = _interests_matrix_with_loc(location)
    df_profile_t = df_new[df_new.columns[2:]]

    index = _index_of(id, df_new)

    profile_d = distance.pdist(df_profile_t, metric='matching')
    profile_D = distance.squareform(profile_d)
    p_r = profile_D[index].tolist()
    del profile_d
    del profile_D
    gc.collect()

    sim = pd.Series(p_r)
    df_new['count_of_interests'] = (1 - sim.values) * len(df_profile_t.columns)
    df_new['count_of_interests'] = df_new['count_of_interests'].astype(int)
    List = [i[0] for i in sorted(enumerate(p_r), key=lambda x:x[1])][0:21]

    return df_new.ix[List][['account_id', 'count_of_interests']]#['V2']

# try:
#     print interests_sim_with_loc(28071, 'New York')
# except:
#     print "error!"
print interests_sim_with_loc(28071, 'New York')

Empty DataFrame
Columns: [account_id, location, Art, Comedy, Dance, Dating, Fashion, Film, Fitness, Food, Museums, Music, Nightlife, Sports, Theater, Wellness, Writing]
Index: []


IndexError: list index out of range

In [ ]:
def events_sim_with_loc(id, location):
    df_new = _events_matrix_with_loc(location)
    df_event_t = df_new[df_new.columns[2:]]

    index = _index_of(id, df_new)

    df_event_t = df_event_t.div(df_event_t.sum(axis=1), axis=0)

    # event_D = np.corrcoef(df_event_t)
    # e_r = event_D[index].tolist()
    # sim = pd.Series(e_r)
    # df_new['similarity_percentage'] = sim.values
    # List = [i[0] for i in sorted(enumerate(e_r), key=lambda x:x[1], reverse=True)][0:21]
    # print event_D.shape

    event_d = distance.pdist(df_event_t, metric='correlation')
    event_D = distance.squareform(event_d)
    del df_event_t
    del event_d

    e_r = event_D[index].tolist()
    del event_D

    gc.collect()

    sim = pd.Series(e_r)
    df_new['similarity_percentage'] = 1 - sim.values
    List = [i[0] for i in sorted(enumerate(e_r), key=lambda x:x[1])][0:21]

    return df_new.ix[List]#[['account_id', 'email', 'similarity_percentage']]#['V2']

events_sim_with_loc(28071, 'New York')

In [ ]:
def _request_data(uri):
    print("Sending request to:", uri)
    request = Request(uri)
    data = json.loads(urlopen(request).read())
    return data


def graph(uri):
    hash = dict()
    connections = _request_data(uri)
    for connection in connections:
        account_id = int(connection['account_id'])
        requestor_id = int(connection['requestor_id'])

        if account_id in hash.keys():
            hash[account_id]['connections'].append(requestor_id)
        else:
            connections1 = []
            connections1.append(requestor_id)
            hash[account_id] = {'connections': connections1}

        if requestor_id in hash.keys():
            hash[requestor_id]['connections'].append(account_id)
        else:
            connections2 = []
            connections2.append(account_id)
            hash[requestor_id] = {'connections': connections2}

    return hash


def _neighbours(graph, user_id):
    if user_id in graph.keys():
        return graph[user_id]['connections']
    else:
        return set()


def bfs(graph, user_id):
    count = 0
    commons = {}
    visited, queue = set(), []
    visited.add(user_id)
    friends = _neighbours(graph, user_id)
    queue = _enqueue(queue, visited, friends, 0)

    while queue:
        current = queue.pop(0)

        if current['id'] not in visited:
            count += 1
            visited.add(current['id'])
            connections = _neighbours(graph, current['id'])
            if connections is not None:
                queue = _enqueue(queue, visited, connections,
                                 current['level'] + 1)

                if current['id'] not in friends:
                    mutuals = _common_friends(graph, user_id, current['id'])
                    commons[current['id']] = {
                        # 'email': graph[current['id']]['email'],
                        'parent': user_id,
                        # 'commons': mutuals,
                        'level': current['level'],
                        'num_of_commons': len(mutuals)
                    }
    print count
    return sorted(commons.items(), key=lambda x: (x[1]['num_of_commons'], -x[1]['level']), reverse=True)[0:20]


def _common_friends(graph, user1, user2):
    a = _neighbours(graph, user1)
    b = _neighbours(graph, user2)

    return set(a).intersection(b)


def _enqueue(queue, visited, arr, level):
    for i in arr:
        if i not in visited:
            queue.append({'id': i, 'level': level})
    return queue


In [ ]:
CONNECTIONS_URI = "http://0.0.0.0:3000/api/v4/re/connections"
g = graph(CONNECTIONS_URI)
bfs(g, 28071)